In [1]:
import os
import torch
import scipy.io
import matplotlib.pyplot as plt
import cvxpy as cp
import seaborn as sns
import numpy as np
import time

os.chdir('..')
import data_processing.preprocessing as preprocessing
from config import left_cut, right_cut
from utils import beerlamb_multi
import utils

sns.set()

In [4]:
path_old = "dataset/miniCYRIL-Piglet-Data/"
path = "/Users/boeykaizhe/Documents/TUM/IDP2/miniCYRIL-Piglet-Data"
folder = os.listdir(path_old)
folder = [i for i in folder if "lwp" in i or "LWP" in i]
dic = {}
for i in folder:    
    files = os.listdir(path_old+i)
    files = [i for i in files if ".mat" in i]
    files = [i for i in files if not any(excluded in i for excluded in 
                    ["Results", "DarkCount", "ref", "output", "markcope", "Abs", "lwp"])]
    if files == []: continue
    dic[i] = files

In [6]:
def optimisation(spectr1, spectr2, wavelengths):
    m = 4  # number of parameters (from 2 to 6)
    #np.random.seed(1)
    molecules, x = preprocessing.read_molecules(left_cut, right_cut, wavelengths)
    y_hbo2_f, y_hb_f, y_coxa, y_creda, _, _ = molecules
    M = np.transpose(np.vstack((np.asarray(y_hbo2_f),
                                np.asarray(y_hb_f),
                                np.asarray(y_coxa),
                                np.asarray(y_creda))))
    
    b = spectr2 / spectr1
    b = np.log(1 / np.asarray(b))  # see the writting above (we took log of I_R and there was also minus that went to the degree of the logarithm)
    X = cp.Variable((m, len(b)))
    b = np.swapaxes(b, 0, 1)
    
    print(M.shape, X.shape, b.shape)
     
    objective = cp.Minimize(cp.sum_squares(M @ X - b))
    constraints = [cp.abs(X[2]+X[3])<=0.01]
    prob = cp.Problem(objective, constraints)
    
    start = time.time()
    result = prob.solve()
    print("Time:", time.time() - start)

    return -X.value, b

In [9]:
def get_dataset(spectr, dark_full, white_full, wavelengths, minimum, maximum, pig, date):
    ref_spectr = (spectr[:, minimum] - dark_full[:, 0]) / (white_full[:, 0] - dark_full[:, 0])
    ref_spectr[ref_spectr <= 0] = 0.0001

    spectra_list = []
    coef_list = []

    #if i not in [100,200,400,2000]: continue
    comp_spectr = np.array([(spectr[:, i] - dark_full[:, 0]) / (white_full[:, 0] - dark_full[:, 0]) for i in range(minimum+1,maximum+1)])
    comp_spectr[comp_spectr <= 0] = 0.0001

    #comp_spectr = np.array(comp_spectr.tolist() * 11)

    coef_diff, spect_diff = optimisation(ref_spectr, comp_spectr, wavelengths)

    spectra_list.append(spect_diff)
    coef_list.append(coef_diff)
    utils.save_optimization_data(ref_spectr, spectra_list, coef_list, str(pig)+'_'+str(date))

In [10]:
for pig in dic.keys():
    for index, date in enumerate(dic[pig]):  
        print(pig, date)
        img = scipy.io.loadmat(path_old + pig + '/' + date)

        wavelengths = img['wavelengths'].astype(float)
        white_full = img['refIntensity'].astype(float)
        dark_full = img['darkcount'].astype(float)
        spectr = img['spectralDataAll'].astype(float)

        idx = (wavelengths >= left_cut) & (wavelengths <= right_cut)
        idx = idx.flatten()
        original_true_count = np.count_nonzero(idx)
        target_true_count = 121

        # Find the indices of True values
        true_indices = np.where(idx)[0]
        additional_false_count = original_true_count - target_true_count

        # Randomly sample additional_false_count indices to set to False
        indices_to_set_false = np.random.choice(true_indices, size=additional_false_count, replace=False)
        idx[indices_to_set_false] = False
        idx = np.expand_dims(idx, axis=1)

        wavelengths = wavelengths[idx]
        spectr = spectr[idx.squeeze()]
        white_full = white_full[idx.squeeze()]

        print(white_full.shape, dark_full.shape, wavelengths.shape, spectr.shape)  
        minimum, maximum = None, None
              
        # fig, (ax, ax1, ax2) = plt.subplots(ncols=3, figsize=(12, 4))
        for i in range(spectr.shape[1]):
            spectr_1 = (spectr[:, i] - dark_full[:, 0]) / (white_full[:, 0] - dark_full[:, 0])
            dif = max(spectr_1) - min(spectr_1)
            if dif >= 0.1:
        #         ax.plot(wavelengths, spectr_1)
        #         ax.set_xlabel("Wavelength", fontsize=15)
        #         ax.set_title("Base Spectrogram", fontsize=15)
                minimum = i
                break

        for i in reversed(range(spectr.shape[1])):
            spectr_2 = (spectr[:, i] - dark_full[:, 0]) / (white_full[:, 0] - dark_full[:, 0])
            dif = max(spectr_2) - min(spectr_2)
            if dif >= 0.1:
        #         ax1.plot(wavelengths, spectr_2)
        #         ax1.set_xlabel("Wavelength", fontsize=15)
        #         ax1.set_title("Hypoxia Spectrogram", fontsize=15)
                maximum = i
                break

        # spectr_1[spectr_1 <= 0] = 0.0001
        # spectr_2[spectr_2 <= 0] = 0.0001
        # ax2.plot(wavelengths, spectr_2 / spectr_1)
        # ax2.set_xlabel("Wavelength", fontsize=15)
        # ax2.set_title("Diff Spectrogram", fontsize=15)
        
        print("Min and Max:", minimum, maximum)
        if minimum is not None and maximum-minimum >= 100: get_dataset(spectr, dark_full, white_full, wavelengths, minimum, maximum, pig, index)
        else: print("skipped this sample")
        print()

LWP495 LWP495_Ws_03Apr2017_12  46.mat
(121, 1) (1, 1) (121,) (121, 63)
Min and Max: 62 62
skipped this sample

LWP495 LWP495_Ws_03Apr2017_12  34.mat
(121, 1) (1, 1) (121,) (121, 4)
Min and Max: 3 3
skipped this sample

LWP495 LWP495_Ws_03Apr2017_12  29.mat
(121, 1) (1, 1) (121,) (121, 11)
Min and Max: None None
skipped this sample

LWP495 LWP495_Ws_03Apr2017_14  23.mat
(121, 1) (1, 1) (121,) (121, 8772)
Min and Max: 132 8771
(121, 4) (4, 8639) (121, 8639)
Time: 10.826961755752563

LWP492 LWP492_Ws24hr_14Mar2017.mat
(121, 1) (1, 1) (121,) (121, 11)
Min and Max: None None
skipped this sample

LWP492 LWP492_Ws_13Mar2017.mat
(121, 1) (1, 1) (121,) (121, 9353)
Min and Max: 179 9352
(121, 4) (4, 9173) (121, 9173)
Time: 10.813344955444336

LWP493 LWP493_Ws_20Mar2017.mat
(121, 1) (1, 1) (121,) (121, 5)
Min and Max: None None
skipped this sample

lwp494 LWP494_Ws_27Mar2017_17  21.mat
(121, 1) (1, 1) (121,) (121, 9879)
Min and Max: 76 9424
(121, 4) (4, 9348) (121, 9348)
Time: 12.367353200912476


In [9]:
#os.mkdir('/Users/boeykaizhe/Documents/TUM/IDP2/IDP_Boey/dataset/piglet_diffs/LWP495_3')
